In [1]:
import io
import glob
import pandas as pd
import numpy
import collections
from csv import reader

In [15]:
#pour lister les absolute path des fichiers à traiter
corpusde = glob.glob("/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/*.txt")
#print(corpusfr)
len(corpusde) #on vérifie combien il y a de fichiers

275

In [16]:
#pour lister les textes en virant les scories qui troublent les noms de colonnes
corpusde_propre =[]
var = []
k=0
while k<len(corpusde):
    var.append(corpusde[k].replace('/home/angelique/Documents/these/corpus/corpus/corpus_total/corp_langue/fr/',''))
    corpusde_propre.append(var[k].replace('.txt',''))
    k+=1
#print(corpusde_propre)

len(corpusde_propre)
corpusde_propre[:10]

['rev_arch1872_2',
 'rev_arch1914_2',
 'BCH1886',
 'BCH1896',
 'letronne1885_corr',
 'seure1906_corr',
 'rev_arch1884_1',
 'ridder1901_corr',
 'rev_arch1877_1',
 'rayet1884_2_corr']

In [7]:
lieuxdic = pd.read_csv("/home/angelique/Documents/these/reperage_EN/lieux_de.csv", sep='\t')
lieuxdic = lieuxdic.drop_duplicates(keep='first')
lieuxdic.head

<bound method NDFrame.head of          lieux
0      Athènes
1        Argos
2      Éleusis
3      Eleusis
4      Pergame
5     Tirynthe
6       Sparte
7       Éphèse
8       Ephèse
9       Ephese
10      Priène
11    Épidaure
12    Epidaure
13        Élis
14        Elis
15         Kos
16  Erechtéion
17  Erechteion
18  Érechteion
19       Milet
20    Syracuse
21     Érétrie
22     Erétrie
23      Mégare
24   Orchomène
25       Samos
26     Olympia
27     Olympie>

In [5]:
lieuxdic = ['Olympia']

In [8]:
listelieux = [] #création d'une liste de noms à insérer plus tard dans les résultats. On prend le temps de supprimer des \n fâcheux
with open("/home/angelique/Documents/these/reperage_EN/lieux_de.csv", 'r') as f:
    for line in f:
        v = str(line)
        v = v.replace('\n','')
        listelieux.append(v)
listelieux.pop(0) #on supprime l'index du tableau puisqu'on remettra encore un nouvel index dans le tableau des résultats
listelieux = list(dict.fromkeys(listelieux))

In [9]:
 #création d'un hashset avec tous les noms
lieux = set((lieuxdic['lieux'].values)) 

In [10]:
def reperagelieux(listeLieux, corpusde_propre):
    results = pd.DataFrame()
    k=0
    ponctuation = [',','.',':',';','!','?','"','»',')','(','„','»','«','”']
    listeLieux = [[] for i in range(len(corpusde_propre))] #on fait une liste de liste. chaque liste contiendra le nom cherché puis le nombre d'occurence de chaque nom
    results = pd.DataFrame() #on crée un df qui contiendra les résultats
    results.insert(loc = 0, column='lieux', value = listelieux) #on met dans le tableau des résultats la première colonne qui est les noms de personnage
    while k < len(corpusde_propre):
        infile = open(corpusde[k],encoding = 'utf-8')
        #print(corpus[k])
        text = infile.read()
        infile.close()
        for punct in ponctuation:
            text = text.replace(punct, ' '+punct)
        #text = text.title() #on fait en sorte que tous les mots commencent par une majuscule (comme dans la liste des noms qu'on cherche) en fait non sinon on a du bruit comme des cons
        texte_liste = text.split() #On transforme le texte en liste de mots séparés par des espaces
        texte_lieux = [token for token in texte_liste if token in lieux] #on stripe les textes des mots qui ne font pas partie de ma liste de noms (but alléger le code)
        compte = collections.Counter(texte_lieux) #dictionnaire avec en clé un nom et en valeur le support associé (la fréquence quoi, on parle l'académique ici)
        #print(compte.most_common(5)) #on lui fait imprimer les compteurs des 5 max de la ligne
        liste_compte = [compte[mot] for mot in lieuxdic['lieux'].values] #liste compréhension qui entre une colonne avec les comptages
        results.insert(loc = k+1,column = corpusde_propre[k], value = liste_compte)
        k+=1
    return results

In [17]:
listeLieux = []

results = reperagelieux(listeLieux, corpusde_propre) 

/home/angelique/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
results['freq']=results.sum(axis=1) #on crée une colonne fréquence avec la somme
results = results.sort_values('freq',ascending=False) #on classe par ordre décroissant
#on se prend un performance warning mais on s'en fiche ça marche

<ipython-input-18-a61a6c6fca6c>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results['freq']=results.sum(axis=1) #on crée une colonne fréquence avec la somme


In [19]:
results.head()

,lieux,rev_arch1872_2,rev_arch1914_2,BCH1886,BCH1896,letronne1885_corr,seure1906_corr,rev_arch1884_1,ridder1901_corr,rev_arch1877_1,...,rev_arch1906_2,rev_arch1894_2,BCH1877,daremberg4_1_1873,rev_arch1898_1,gaz_arch1886,rev_arch1903_2,collignon1898_corr,wagnon1885_corr,freq
0,Athènes,22,5,21,79,40,2,10,7,5,...,14,19,54,22,61,10,16,1,4,6108
4,Pergame,2,2,15,9,9,0,5,0,0,...,9,2,3,6,2,10,13,0,0,2019
6,Sparte,3,0,3,37,9,0,1,0,0,...,4,2,4,8,1,0,0,0,3,1135
25,Samos,5,1,5,8,8,0,3,7,0,...,1,10,1,3,0,5,0,0,0,1040
27,Olympie,2,1,3,9,5,0,2,1,3,...,2,5,0,5,0,4,5,1,4,999


In [104]:
results2 = results[results.freq > 99] #on crée un second dataframe qui ne contient que les noms dont la fréquence est supérieure ou égale à 100
results2.head

<bound method NDFrame.head of      toponymes  jdi1909  bwpr1887  schliemann1874  sauer1887  bwpr1893  \
7582       Der      190        18              66         19        73   
137      Athen       50        29              34         17        33   
7170      Auch       61         7              65         15        14   
1493       Rom       38         1               1          0        92   
224     Berlin       44        22               1          7        70   
...        ...      ...       ...             ...        ...       ...   
4456   Potsdam        0         0               0          0         0   
7543    Byblos        0         0               0          0         0   
3859  Akanthos        0         0               0          0         0   
831      Kairo        4         0               0          0         0   
725     Ikaria        0         0               0          0         0   

      bwpr1891  mit_dai_ath1882  jdi1892  jdi1898  ...  winnefeld1887  \
7582    

In [102]:
results2 = results2.transpose(copy=False)

In [20]:
results.to_csv(r'/home/angelique/Documents/these/reperage_EN/geog/export_python/results_lieux_fr_total_230131.csv') #on exporte